In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas 
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from math import log
import math

In [3]:
#leitura da base e definição dos campos utilizados
baseTotalAtributos = pandas.read_csv (r"C:\Users\gerso\Desktop\EP2_ia2019\dataset_5secondWindow%5B3%5D.csv", usecols =  ["time","android.sensor.accelerometer#mean","android.sensor.linear_acceleration#mean","android.sensor.orientation#mean","speed#mean"])
baseTotalClasse = pandas.read_csv (r"C:\Users\gerso\Desktop\EP2_ia2019\dataset_5secondWindow%5B3%5D.csv", usecols = ["target"])

print(baseTotalAtributos.columns.values)

['time' 'android.sensor.accelerometer#mean'
 'android.sensor.linear_acceleration#mean'
 'android.sensor.orientation#mean' 'speed#mean']


In [4]:

#normalização linear dos dados 
def normalizacaoLinear(baseTotalAtr):
    colunas =  list(baseTotalAtr.columns)

    baseTotalAtributosNorm = baseTotalAtr.copy()
    baseTotalAtributosNorm[colunas] = baseTotalAtr[colunas].apply(minmax_scale)
    
    return baseTotalAtributosNorm

In [6]:

atributosNormalizados = normalizacaoLinear(baseTotalAtributos)

#particionamento da base 70 % para o conjunto de teste e 30 % para o conjunto de treinamento
baseTreinoNorm,  baseTesteNorm , classeTreinoNorm, classeTesteNorm = train_test_split(atributosNormalizados, baseTotalClasse, test_size = 0.3 , train_size = 0.7, random_state = 0)
baseTreinoNaoNorm,  baseTesteNaoNorm , classeTreinoNaoNorm, classeTesteNaoNorm = train_test_split(baseTotalAtributos, baseTotalClasse, test_size = 0.3 , train_size = 0.7, random_state = 0)


In [7]:
#discretização dos dados
def discretizacaoBase(baseAtributos):
    
    baseDiscretizada = pandas.DataFrame()
    for coluna in baseAtributos.columns.values:
        baseDiscretizada[coluna] = pandas.cut(baseAtributos[coluna],bins = 5,labels= range(5))  
    return baseDiscretizada

In [74]:
def CalculaEntropia (classes):
    results , counts = np.unique(classes , return_counts = True) #conta a quantidade de de domínio presente em uma coluna, no caso essa coluna é a classe 
    
    entropia = 0.0
    for r in range(len(results)):
        p = float (counts[r])/len(classes)
        entropia = entropia-p*log(p,2)
  
    return entropia
     

In [82]:
class Decisionnode:
    def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
        self.col = col
        self.value = value
        self.results = results
        self.tb = tb
        self.fb = fb

In [145]:
def divideset(classes,data,coluna,valor):
    
    set1 = data[data[coluna]==valor] #seleciona apenas os registros com a coluna IGUAL ao valor especificado
    set2 = data[data[coluna]!=valor] #seleciona apenas os registros com a coluna DIFERENTE ao valor especificado
    
    mergedSet1 = set1.merge(classes, left_index=True, right_index=True, how='inner')
    mergedSet2 = set2.merge(classes, left_index=True, right_index=True, how='inner')
    
    
    return mergedSet1,mergedSet2

In [249]:
def GanhoInformacao(atributo, baseAtributos, entropia):
    #dominios = base[atributo].unique()
    dominios , qtdDominios = np.unique(baseAtributos[atributo], return_counts=True) 
    
    soma = 0
    
    for dominio in dominios: 
        set1 = baseAtributos[baseAtributos[atributo]== dominio] # pegar todos os registros que possuam o valor "dominio"  
        soma += (len(set1)/len(baseAtributos))*CalculaEntropia(set1['target']) #calcula os componentes somatório do ganho de informação
    
    return(entropia - soma)
           

In [298]:
def arvoreDecisao(base_atributos_treino):
    
    current_score = CalculaEntropia(base_atributos_treino['target'])
    
    if (len(base_atributos_treino) == 0  or current_score == 0.0):
        return
    
    colunas = base_atributos_treino.keys()[:-1]
    classe_ganho_inf = {}
    
    for atributo in (colunas):
        classe_ganho_inf[atributo] = GanhoInformacao(atributo,base_atributos_treino,current_score)
    atributo_escolhido = max(classe_ganho_inf , key = classe_ganho_inf.get)
    maior_ganho_info = classe_ganho_inf[atributo_escolhido]
    classe_ganho_inf.clear()
    
    dominios = base_atributos_treino[atributo_escolhido].unique().sort_values()
    print(atributo_escolhido,maior_ganho_info,current_score,dominios)
    
    for dominio in dominios :
        subArvore = base_atributos_treino[base_atributos_treino[atributo_escolhido]== dominio]
        arvoreDecisao(subArvore)
   
    
    

In [299]:

bDiscretizada = discretizacaoBase(baseTreinoNorm) 
mergeDados = bDiscretizada.merge(baseTotalClasse, left_index=True, right_index=True, how='inner') # fazer um join com a "base" de classes para identificar a respectiva classe para usar no cálculo da entropia
     
arvoreDecisao(mergeDados)

speed#mean 0.26195908512060706 2.3218794058760355 [0, 1, 2, 3, 4]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
android.sensor.linear_acceleration#mean 0.2623338546781273 2.2831981449861503 [0, 1, 2, 3, 4]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
time 0.17212432805374167 2.2635672226246832 [0, 1, 2, 3, 4]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
android.sensor.orientation#mean 0.11124298006093536 2.2143441644134176 [0, 1, 2, 3, 4]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
android.sensor.accelerometer#mean 0.021666034456102157 2.1399317111376885 [0, 1]
Categories (2, int64): [0 < 1]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): 

time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [0]
time 0.0 1.8467341583372252 [0]
Categories (1, int64): [

KeyboardInterrupt: 

In [ ]:

    dominios = base_atributos_treino[atributo_escolhido].unique()
    
    for
    
    subArvore = base_atributos_treino[base_atributos_treino[atributo_escolhido]== dominio]
    arvoreDecisao(base_atributos_treino)